# 進出場策略 (Entry & Exit Strategies)

## 看一下單一股票的進出場狀況

In [ ]:
import ffn

%matplotlib inline

import pandas_datareader.data as web
import pandas as pd
from pandas import Series, DataFrame

import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime

In [ ]:
# 讀取從指定日期之後的股價資訊
df = web.DataReader("TSLA", 'google', datetime(2016,8,1))

In [ ]:
# 計算均線
df['20d'] = pd.Series.rolling(df['Close'], window=20).mean()
df['60d'] = pd.Series.rolling(df['Close'], window=60).mean()

In [ ]:
df[['Close','20d','60d']].plot(grid=True, figsize=(10,8))

In [ ]:
# Donchian Channel
df['20d_high'] = pd.Series.rolling(df['Close'], window=20).max()
df['10d_low'] = pd.Series.rolling(df['Close'], window=10).min()

In [ ]:
df[['Close','20d_high','10d_low']].plot(grid=True, figsize=(10,8))

## 策略

進出點的計算，可以參考：

https://www.quantstart.com/articles/Backtesting-a-Moving-Average-Crossover-in-Python-with-pandas

In [ ]:
# 第一個策略
def strategy1(df):
    # 計算均線
    df['20d'] = pd.Series.rolling(df['Close'], window=20).mean()
    df['60d'] = pd.Series.rolling(df['Close'], window=60).mean()
    
    df['positions'] = np.where(df['20d']-df['60d'] > 0, 1.0, 0.0)
    df['signals'] = df['positions'].diff()
    df[['signals', 'positions']].plot(subplots = True, ylim=(-1.1, 1.1), figsize = (10, 8))

In [ ]:
# 第二個策略
def strategy2(df):
    # 計算均線
    df['20d'] = pd.Series.rolling(df['Close'], window=20).mean()
    df['60d'] = pd.Series.rolling(df['Close'], window=60).mean()
    
    # 判斷均線向上或向下
    df['20d_diff'] = np.round(df['20d'].diff(), 2)
    df['60d_diff'] = np.round(df['60d'].diff(), 2)
    
    df['positions'] = np.where(np.logical_and(df['20d']-df['60d'] > 0, df['20d_diff'] > 0), 1.0, 0.0)
    df['signals'] = df['positions'].diff()
    df[['signals', 'positions']].plot(subplots = True, ylim=(-1.1, 1.1), figsize = (10, 8))

In [ ]:
# 第三個策略
def breakout(df):
    # Donchian Channel
    df['20d_high'] = pd.Series.rolling(df['Close'], window=20).max()
    df['10d_low'] = pd.Series.rolling(df['Close'], window=10).min()

    has_position = False
    df['signals'] = np.zeros(np.size(df['Close']))
    for t in range(2, df['signals'].size):
        if df['Close'][t] > df['20d_high'][t-1]:
            if not has_position:
                df.loc[df.index[t], 'signals'] = 1.0
                has_position = True
        elif df['Close'][t] < df['10d_low'][t-1]:
            if has_position:
                df.loc[df.index[t], 'signals'] = -1.0
                has_position = False

    df['positions'] = df['signals'].cumsum()

    df[['signals', 'positions']].plot(subplots = True, ylim=(-1.1, 1.1), figsize = (10, 8))

In [ ]:
def apply_strategy(strategy, df):
    return strategy(df)

In [ ]:
apply_strategy(breakout, df)

In [ ]:
fig = plt.figure()
#fig.patch.set_facecolor('white')     # Set the outer colour to white
ax1 = fig.add_subplot(111,  ylabel='Price in $')
    
df['Close'].plot(ax=ax1, color='gray', lw=1., figsize=(10,8))

# Plot the "buy" trades
ax1.plot(df.ix[df.signals == 1.0].index,df['Close'][df.signals == 1.0],'^', markersize=10, color='r')

# Plot the "sell" trades
ax1.plot(df.ix[df.signals == -1.0].index, df['Close'][df.signals == -1.0], 'v', markersize=10, color='k')

## 計算Sharpe Ratio

In [ ]:
dailyRet = df['Close'].pct_change()

In [ ]:
#假設無風險利率為 4%
#假設一年有252個交易日
excessRet = (dailyRet - 0.04/252)[df['positions']==1.0]

sharpeRatio = np.sqrt(252.0)*np.mean(excessRet)/np.std(excessRet)

In [ ]:
sharpeRatio

## 計算MaxDD跟MaxDDD

In [ ]:
cumRet = np.cumprod(1 + excessRet)

In [ ]:
cumRet.plot(style='ro-')

In [ ]:
def DrawDownAnalysis(cumRet):
    dd_series = ffn.core.to_drawdown_series(cumRet)
    dd_details = ffn.core.drawdown_details(dd_series)
    return dd_details['drawdown'].min(), dd_details['days'].max()

In [ ]:
DrawDownAnalysis(cumRet)